## PARTE 2 - ¿De qué manera se ha visto afectado el consumo y la producción de F&H durante la pandemia con respecto a años anteriores?

Los datos tanto de consumo como de producción se han obtenido del MAPA en los enlaces que se aportan a continuación. La obtención de todos los datos de consumo desde el año 2013 al 2020 se ha realizado mediante scrapping con Selenium. Solo se contaba con datos anuales de producción, por lo que el análisis sera anual para 2018-2019-2020

Fuente de los datos: 
- Datos consumo : https://www.mapa.gob.es/app/consumo-en-hogares/consulta11.asp
- Datos producción : https://www.mapa.gob.es/es/estadistica/temas/estadisticas-agrarias/agricultura/avances-superficies-producciones-agricolas/

In [1]:
# Se cargan las liberías
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
# Se cargan los dos datasets (producción y consumo)
df_produccion = pd.read_csv("Datos_produccion.csv", encoding = 'utf-8-sig')
df_consumo = pd.read_csv("consumoF&H_Limpio_2.csv", encoding = 'utf-8-sig')

In [3]:
# Se filtra el consumo nacional para compararlo con la producción nacional
df_consumo = df_consumo[df_consumo['CCAA'] == 'Total Nacional']

In [4]:
# Calcular el valor medio de todos los meses para 2018 y 2019 y rellenar con ese dato Diciembre (mes faltante)
value2018 = df_consumo.groupby(['Año','Producto','CCAA'], as_index = False)['Volumen (miles de kg)','Valor (miles de €)'].agg('mean')
value2018['Mes'] = 12
value2018['Fecha'] =  pd.to_datetime(value2018['Año'].map(str) + '-' + value2018['Mes'].map(str)).dt.strftime('%m-%Y')
value2018 = value2018[value2018['Año'] != 2020]
# Se añade al dataset de consumo
df_consumo = pd.concat([df_consumo, value2018], axis= 0)
df_consumo[df_consumo['Fecha'] == '11-2020']
# Se aáde el mes de diciembre también a los productos que tienen datos hasta noviemb
value_2020 = df_consumo[df_consumo['Fecha'] == '11-2020'].copy()
value_2020['Mes'] = 12
value_2020['Fecha'] = '12-2020'
df_consumo = pd.concat([df_consumo, value_2020], axis= 0)

<ipython-input-4-c19235d6fc84>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  value2018 = df_consumo.groupby(['Año','Producto','CCAA'], as_index = False)['Volumen (miles de kg)','Valor (miles de €)'].agg('mean')


In [5]:
# Se saca una lista de todos los productos que tienen datos hasta noviembre
alimentos = value_2020['Producto'].tolist()

In [6]:
# Se construte un dataframe con los productos que no están en esa lista y como solo hay datos hasta junio se completa cogiendo los datos hasta diciembre del 2019
df_2020 = df_consumo[~df_consumo['Producto'].isin(alimentos)]
fechas = ['07-2019','08-2019','09-2019','10-2019','11-2019','12-2019']
df_2020 = df_2020[df_2020['Fecha'].isin(fechas)]
values2020mj = df_2020.copy()
values2020mj['Año'] = 2020
values2020mj['Fecha'] = pd.to_datetime(values2020mj['Año'].map(str) + '-' + values2020mj['Mes'].map(str)).dt.strftime('%m-%Y')
df_consumo = pd.concat([df_consumo, values2020mj], axis= 0)

In [7]:
value2018['Mes'] = 12
value2018['Fecha'] =  pd.to_datetime(value2018['Año'].map(str) + '-' + value2018['Mes'].map(str)).dt.strftime('%m-%Y')
value2018 = value2018[value2018['Año'] != 2020]
df_consumo = pd.concat([df_consumo, value2018], axis= 0)

In [8]:
# Se ordenan los datos por año y mes
df_consumo.sort_values(by = ['Año','Mes'], inplace = True)

In [9]:
# Se crea el dataset df_final para calcular el consumo total por producto y año y poder compararlo con la producción total
df_consumofinal = pd.DataFrame(df_consumo.groupby(['Año','Producto'], as_index = False)[['Volumen (miles de kg)','Valor (miles de €)']].sum())

In [10]:
# Se renombran las columnas del dataset de producción para quitar el año 
df_produccion2018 = df_produccion[['Cultivos','Producción 2018 (toneladas)']].rename(columns={'Producción 2018 (toneladas)':'Producción(toneladas)'})
df_produccion2019 = df_produccion[['Cultivos','Producción 2019 (toneladas)']].rename(columns={'Producción 2019 (toneladas)':'Producción(toneladas)'})
df_produccion2020 = df_produccion[['Cultivos','Producción 2020 (toneladas)']].rename(columns={'Producción 2020 (toneladas)':'Producción(toneladas)'})

In [11]:
# Se parte el dataset de producción por años para que tenga el mismo formato que el dataset de consumo
df_produccion2018['Año'] = 2018
df_produccion2019['Año'] = 2019
df_produccion2020['Año'] = 2020

In [12]:
# Se concatenan los 3 datasets de forma que en el mismo está la producción por producto y año
df_produccion = pd.concat([df_produccion2018,df_produccion2019,df_produccion2020],axis = 0)

In [13]:
# Se unifican los datasets de producción y consumo por el campo producto y año
#pd.set_option('display.float_format', lambda x: '%.3f' % x)
df_cons_prod = df_consumofinal.merge(df_produccion, left_on=['Producto','Año'], right_on=['Cultivos','Año'], how='inner')
df_cons_prod.drop(columns = 'Cultivos', inplace=True)
df_cons_prod.rename(columns = {'Volumen (miles de kg)' : 'Volumen(toneladas)'}, inplace = True)
df_cons_prod[['Volumen(toneladas)','Valor (miles de €)']] = df_cons_prod[['Volumen(toneladas)','Valor (miles de €)']].apply(np.int64)

In [14]:
# Dataset consumo (en toneladas y en miles de euros) y producción (toneladas)
df_cons_prod

,Año,Producto,Volumen(toneladas),Valor (miles de €),Producción(toneladas)
0,2018,AGUACATE,59217,201301,89600
1,2018,AJOS,39614,154406,273500
2,2018,ALBARICOQUES,48628,96658,176300
3,2018,ALCACHOFAS,70360,113595,195600
4,2018,BERENJENAS,80471,119022,238300
...,...,...,...,...,...
94,2020,SANDIA,437694,388629,1244200
95,2020,TOMATES,682010,1192852,4238400
96,2020,TOTAL PATATAS,1407039,1367097,2139900
97,2020,UVAS,111434,311385,297800


In [15]:
# Se guardan el dataset de consumo y producción
# df_cons_prod.to_csv("Consumo_Producción.csv", encoding= "utf-8-sig")

### REPRESENTACIÓN DEL CONSUMO Y LA PRODUCCIÓN

A continuación, se representa el consumo y la producción por cada año y producto de forma que se puedan extraer conclusiones de la relación existente entre ambas variables.

In [16]:
from IPython.display import IFrame
IFrame("https://app.powerbi.com/view?r=eyJrIjoiZGVkNDBhMzUtYTU4YS00MWVkLWEwNjYtYzRlY2ZiMTVmODhhIiwidCI6IjZhZmVhODVkLWMzMjMtNDI3MC1iNjlkLWE0ZmIzOTI3YzI1NCIsImMiOjl9",height = 550, width = 1000) 

*En el reporte se observa por un lado la evolución en los últimos 3 años de la producción y el consumo por producto (frutas y hortalizas). En la parte de abajo se representan 3 gráficos en % del consumo anual (tanto en toneladas como en miles de €) y la producción para poder evaluar su evolución en los últimos 3 años.*

Para visualizar el análisis detallado, acudir a: https://datacrop.es/reto-analisis-1/